In [1]:
# Load the data of estimated flood map (EFM)
import os
import fiona
import pyproj
import numpy as np
import rasterio
import rasterio.mask
import rasterio.features
import rasterio.warp
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, MultiPolygon, LineString

# Washington related data dir
EFM_dir = 'data_pro/EFM/Washington_DC.tif'
FFM_dir = 'data/raw/flood/FEMA/Washington Flood Data from FEMA/110001_20190429/S_FLD_HAZ_AR.shp'

In [2]:
# Check estimated flood map
with rasterio.open(EFM_dir) as src:
    assert src.indexes == (1, )
    
    # Data matrix. **NOTE** its shape is Height(Y)*Width(X)
    data_mat = src.read(1)
    
    # Count different values of data matrix
    values_count = dict()
    for i in range(data_mat.shape[0]):
        for j in range(data_mat.shape[1]):
            if not data_mat[i][j] in values_count: values_count[data_mat[i][j]] = 1
            else: values_count[data_mat[i][j]] += 1
    
    #for k, v in values_count.items():
        #print('value:', k, 'count:', v)

whole_area = values_count[0] + values_count[1]
flood_area = values_count[1]
EFM_ratio = flood_area / whole_area
print('[EFM] Ratio of flood area to whole area:', EFM_ratio)

[EFM] Ratio of flood area to whole area: 0.1859341082912381


In [3]:
# Check FEMA flood map
df = gpd.read_file(FFM_dir)
whole_area = flood_area = 0
for index, row in df.iterrows():
    geom = row['geometry']
    
    # Update whole area
    whole_area += geom.area
        
    # Update flood area (need to specify which area is flooded)
    #if (row['ZONE_SUBTY'] == '0.2 PCT ANNUAL CHANCE FLOOD HAZARD') or (row['ZONE_SUBTY'] == None):
    if (row['ZONE_SUBTY'] != 'AREA OF MINIMAL FLOOD HAZARD'):
        flood_area += geom.area
        
FEMA_ratio = flood_area / whole_area
print('[FFM] Ratio of flood area to whole area:', FEMA_ratio)

print('\nDifference between EFM and FEMA:', FEMA_ratio - EFM_ratio)

[FFM] Ratio of flood area to whole area: 0.19981711829915286

Difference between EFM and FEMA: 0.013883010007914753


# Functionalities for Future Usage

In [ ]:
with rasterio.open(src_dir) as src:
    
    # Some settings. X-axis and Y-axis corresponds width and height respectively.
    res_x, res_y = src.res
    num_y, num_x = src.shape
    assert ((src.bounds.right - src.bounds.left) / res_x == num_x) and ((src.bounds.top - src.bounds.bottom) / res_y == num_y)
    print(num_x, num_y, res_x, res_y)
    
    # Data of a given coordinate
    x, y = 1611028.2,1928599.3
    pixel = src.sample([(x, y)])
    for value in pixel: print(value)
    
    # Longitude and latitude given a coordinate and CRS
    my_proj = pyproj.Proj(src.crs)
    left_bottom = my_proj(src.bounds.left, src.bounds.bottom, inverse=True)
    right_top = my_proj(src.bounds.right, src.bounds.top, inverse=True)
    print(left_bottom, right_top)
    
    # Usually there's only one band
    assert src.indexes == (1, )
    # Data matrix. **NOTE** its shape is Height(Y)*Width(X)
    data_mat = src.read(1)
    